# 🏗️ BigQuery AI: Intelligent Retail Analytics Engine Setup

**Competition Entry**: BigQuery AI - Building the Future of Data
**High-Quality Solution**: Enterprise-Grade Retail Intelligence
**Author**: Senior Data Engineer & AI Architect

---

## 🎯 Overview

This notebook provides a **complete setup guide** for deploying the Intelligent Retail Analytics Engine on Google Cloud Platform. The setup process includes:

1. **🗄️ Dataset Creation** - BigQuery datasets and tables
2. **🤖 Model Setup** - Vertex AI ML models
3. **🔗 Connection Setup** - Vertex AI integration
4. **📊 Data Population** - Sample data and embeddings
5. **⚙️ Configuration** - System configuration files

**Result**: Fully functional BigQuery AI retail analytics system

## 📋 Prerequisites

### Required Google Cloud Setup:
1. **Google Cloud Project** with billing enabled
2. **BigQuery API** enabled
3. **Vertex AI API** enabled
4. **Google Cloud SDK** installed and configured
5. **Project Editor** or **Owner** permissions

### Required Permissions:
- `bigquery.datasets.create`
- `bigquery.tables.create`
- `bigquery.jobs.create`
- `aiplatform.models.*`
- `storage.objects.*`

### Software Requirements:
```bash
# Install Google Cloud SDK
curl https://sdk.cloud.google.com | bash
exec -l $SHELL
gcloud init

# Install Python packages
pip install google-cloud-bigquery google-cloud-aiplatform PyYAML
```

In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🏗️ BigQuery AI: Intelligent Retail Analytics Engine Setup\n",
    "\n",
    "**Competition Entry**: BigQuery AI - Building the Future of Data\n",
    "**High-Quality Solution**: Enterprise-Grade Retail Intelligence\n",
    "**Author**: Senior Data Engineer & AI Architect\n",
    "\n",
    "---\n",
    "\n",
    "## 🎯 Overview\n",
    "\n",
    "This notebook provides a **complete setup guide** for deploying the Intelligent Retail Analytics Engine on Google Cloud Platform. The setup process includes:\n",
    "\n",
    "1. **🗄️ Dataset Creation** - BigQuery datasets and tables\n",
    "2. **🤖 Model Setup** - Vertex AI ML models\n",
    "3. **🔗 Connection Setup** - Vertex AI integration\n",
    "4. **📊 Data Population** - Sample data and embeddings\n",
    "5. **⚙️ Configuration** - System configuration files\n",
    "\n",
    "**Result**: Fully functional BigQuery AI retail analytics system"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📋 Prerequisites\n",
    "\n",
    "### Required Google Cloud Setup:\n",
    "1. **Google Cloud Project** with billing enabled\n",
    "2. **BigQuery API** enabled\n",
    "3. **Vertex AI API** enabled\n",
    "4. **Google Cloud SDK** installed and configured\n",
    "5. **Project Editor** or **Owner** permissions\n",
    "\n",
    "### Required Permissions:\n",
    "- `bigquery.datasets.create`\n",
    "- `bigquery.tables.create`\n",
    "- `bigquery.jobs.create`\n",
    "- `aiplatform.models.*`\n",
    "- `storage.objects.*`\n",
    "\n",
    "### Software Requirements:\n",
    "```bash\n",
    "# Install Google Cloud SDK\n",
    "curl https://sdk.cloud.google.com | bash\n",
    "exec -l $SHELL\n",
    "gcloud init\n",
    "\n",
    "# Install Python packages\n",
    "pip install google-cloud-bigquery google-cloud-aiplatform PyYAML\n",
    "```"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📦 Import required libraries\n",
    "import os\n",
    "import sys\n",
    "import time\n",
    "import logging\n",
    "from pathlib import Path\n",
    "from typing import Dict, List, Optional\n",
    "import yaml\n",
    "\n",
    "# Configure logging\n",
    "logging.basicConfig(\n",
    "    level=logging.INFO,\n",
    "    format='%(asctime)s - %(levelname)s - %(message)s',\n",
    "    handlers=[\n",
    "        logging.FileHandler('retail_analytics_setup.log'),\n",
    "        logging.StreamHandler(sys.stdout)\n",
    "    ]\n",
    ")\n",
    "logger = logging.getLogger(__name__)\n",
    "\n",
    "print(\"✅ Libraries imported successfully!\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔧 Configuration - UPDATE THESE VALUES\n",
    "PROJECT_ID = \"intelligent-retail-analytics\"  # Replace with your actual Google Cloud Project ID\n",
    "DATASET_LOCATION = \"us\"  # Choose: us, eu, asia, etc.\n",
    "\n",
    "print(f\"🔧 Project ID: {PROJECT_ID}\")\n",
    "print(f\"📍 Dataset Location: {DATASET_LOCATION}\")\n",
    "print(\"\\n📝 Make sure to update PROJECT_ID above with your actual Google Cloud Project ID\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🏗️ Setup Class Definition\n",
    "class BigQueryRetailAnalyticsSetup:\n",
    "    \"\"\"Setup and deployment class for the Intelligent Retail Analytics Engine\"\"\"\n",
    "\n",
    "    def __init__(self, project_id: str, dataset_location: str = 'us'):\n",
    "        self.project_id = project_id\n",
    "        self.dataset_location = dataset_location\n",
    "        self.datasets = ['retail_analytics', 'retail_models', 'retail_insights']\n",
    "\n",
    "        # Check if gcloud is available\n",
    "        self._check_gcloud_setup()\n",
    "\n",
    "    def _check_gcloud_setup(self):\n",
    "        \"\"\"Verify Google Cloud SDK setup\"\"\"\n",
    "        try:\n",
    "            import subprocess\n",
    "            result = subprocess.run(['gcloud', '--version'],\n",
    "                                  capture_output=True, text=True, check=True)\n",
    "            logger.info(\"Google Cloud SDK found\")\n",
    "            print(\"✅ Google Cloud SDK found\")\n",
    "        except (subprocess.CalledProcessError, FileNotFoundError):\n",
    "            logger.error(\"Google Cloud SDK not found. Please install and configure gcloud CLI.\")\n",
    "            logger.error(\"Installation: https://cloud.google.com/sdk/docs/install\")\n",
    "            print(\"❌ Google Cloud SDK not found\")\n",
    "            print(\"Installation: https://cloud.google.com/sdk/docs/install\")\n",
    "            sys.exit(1)\n",
    "\n",
    "    def _run_bq_command(self, command: str, description: str) -> bool:\n",
    "        \"\"\"Execute BigQuery command with error handling\"\"\"\n",
    "        try:\n",
    "            logger.info(f\"Executing: {description}\")\n",
    "            import subprocess\n",
    "\n",
    "            # Add project ID to command if not present\n",
    "            if '--project_id' not in command and self.project_id not in command:\n",
    "                command = f\"bq --project_id={self.project_id} {command}\"\n",
    "\n",
    "            result = subprocess.run(command, shell=True, capture_output=True, text=True)\n",
    "\n",
    "            if result.returncode == 0:\n",
    "                logger.info(f\"✅ {description} completed successfully\")\n",
    "                print(f\"✅ {description} completed successfully\")\n",
    "                return True\n",
    "            else:\n",
    "                logger.error(f\"❌ {description} failed\")\n",
    "                logger.error(f\"Error: {result.stderr}\")\n",
    "                print(f\"❌ {description} failed\")\n",
    "                print(f\"Error: {result.stderr}\")\n",
    "                return False\n",
    "\n",
    "        except Exception as e:\n",
    "            logger.error(f\"Exception during {description}: {str(e)}\")\n",
    "            print(f\"❌ Exception during {description}: {str(e)}\")\n",
    "            return False\n",
    "\n",
    "    def _run_gcloud_command(self, command: str, description: str) -> bool:\n",
    "        \"\"\"Execute gcloud command with error handling\"\"\"\n",
    "        try:\n",
    "            logger.info(f\"Executing: {description}\")\n",
    "            import subprocess\n",
    "\n",
    "            result = subprocess.run(command, shell=True, capture_output=True, text=True)\n",
    "\n",
    "            if result.returncode == 0:\n",
    "                logger.info(f\"✅ {description} completed successfully\")\n",
    "                print(f\"✅ {description} completed successfully\")\n",
    "                return True\n",
    "            else:\n",
    "                logger.error(f\"❌ {description} failed\")\n",
    "                logger.error(f\"Error: {result.stderr}\")\n",
    "                print(f\"❌ {description} failed\")\n",
    "                print(f\"Error: {result.stderr}\")\n",
    "                return False\n",
    "\n",
    "        except Exception as e:\n",
    "            logger.error(f\"Exception during {description}: {str(e)}\")\n",
    "            print(f\"❌ Exception during {description}: {str(e)}\")\n",
    "            return False\n",
    "\n",
    "# Initialize setup\n",
    "setup = BigQueryRetailAnalyticsSetup(PROJECT_ID, DATASET_LOCATION)\n",
    "print(\"✅ Setup class initialized successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🗄️ Step 1: Create BigQuery Datasets\n",
    "\n",
    "Create the required BigQuery datasets for the retail analytics system."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🗄️ Create BigQuery Datasets\n",
    "def create_datasets():\n",
    "    \"\"\"Create required BigQuery datasets\"\"\"\n",
    "    logger.info(\"Creating BigQuery datasets...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🗄️ CREATING BIGQUERY DATASETS\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    success_count = 0\n",
    "    for dataset in setup.datasets:\n",
    "        dataset_id = f\"{setup.project_id}:{dataset}\"\n",
    "        command = f\"mk --dataset --location={setup.dataset_location} {dataset_id}\"\n",
    "\n",
    "        if setup._run_bq_command(command, f\"Create dataset {dataset}\"):\n",
    "            success_count += 1\n",
    "        else:\n",
    "            logger.warning(f\"Dataset {dataset} might already exist\")\n",
    "            print(f\"⚠️ Dataset {dataset} might already exist\")\n",
    "\n",
    "    logger.info(f\"Created {success_count}/{len(setup.datasets)} datasets\")\n",
    "    print(f\"\\n📊 Dataset Creation: {success_count}/{len(setup.datasets)} completed\")\n",
    "    return success_count > 0\n",
    "\n",
    "# Run dataset creation\n",
    "create_datasets()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔗 Step 2: Setup Vertex AI Connection\n",
    "\n",
    "Create a Vertex AI connection for BigQuery ML to access Vertex AI models."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔗 Setup Vertex AI Connection\n",
    "def setup_vertex_ai_connection():\n",
    "    \"\"\"Set up Vertex AI connection for BigQuery ML\"\"\"\n",
    "    logger.info(\"Setting up Vertex AI connection...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🔗 SETTING UP VERTEX AI CONNECTION\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    connection_name = \"vertex-connection\"\n",
    "    command = f\"mk --connection --connection_type=CLOUD_RESOURCE --location={setup.dataset_location} {connection_name}\"\n",
    "\n",
    "    if setup._run_bq_command(command, \"Create Vertex AI connection\"):\n",
    "        logger.info(\"Vertex AI connection created successfully\")\n",
    "        print(\"✅ Vertex AI connection created successfully\")\n",
    "        print(\"\\n📝 Note: You may need to grant the BigQuery Connection Service Account access to Vertex AI\")\n",
    "        return True\n",
    "    else:\n",
    "        logger.warning(\"Vertex AI connection setup may have failed\")\n",
    "        print(\"⚠️ Vertex AI connection setup may have failed\")\n",
    "        return False\n",
    "\n",
    "# Run Vertex AI connection setup\n",
    "setup_vertex_ai_connection()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ☁️ Step 3: Enable Required APIs\n",
    "\n",
    "Enable the necessary Google Cloud APIs for the system to function."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ☁️ Enable Required APIs\n",
    "def enable_required_apis():\n",
    "    \"\"\"Enable required Google Cloud APIs\"\"\"\n",
    "    logger.info(\"Enabling required Google Cloud APIs...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"☁️ ENABLING REQUIRED GOOGLE CLOUD APIs\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    apis = [\n",
    "        'bigquery.googleapis.com',\n",
    "        'bigqueryconnection.googleapis.com',\n",
    "        'aiplatform.googleapis.com'\n",
    "    ]\n",
    "\n",
    "    success_count = 0\n",
    "    for api in apis:\n",
    "        command = f\"services enable {api}\"\n",
    "        if setup._run_gcloud_command(command, f\"Enable {api}\"):\n",
    "            success_count += 1\n",
    "\n",
    "    logger.info(f\"Enabled {success_count}/{len(apis)} APIs\")\n",
    "    print(f\"\\n📊 API Enablement: {success_count}/{len(apis)} completed\")\n",
    "    return success_count == len(apis)\n",
    "\n",
    "# Run API enablement\n",
    "enable_required_apis()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📄 Step 4: Execute SQL Implementation\n",
    "\n",
    "Run the main SQL implementation file to create all tables, models, and functions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📄 Execute SQL Implementation\n",
    "def run_sql_file():\n",
    "    \"\"\"Execute SQL file in BigQuery\"\"\"\n",
    "    sql_file_path = \"../retail_analytics_engine.sql\"\n",
    "    \n",
    "    if not Path(sql_file_path).exists():\n",
    "        logger.error(f\"SQL file not found: {sql_file_path}\")\n",
    "        print(f\"❌ SQL file not found: {sql_file_path}\")\n",
    "        return False\n",
    "\n",
    "    logger.info(f\"Executing SQL file: {sql_file_path}\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"📄 EXECUTING SQL IMPLEMENTATION\")\n",
    "    print(\"=\"*60)\n",
    "    print(f\"📁 SQL File: {sql_file_path}\")\n",
    "\n",
    "    command = f\"query --use_legacy_sql=false < {sql_file_path}\"\n",
    "\n",
    "    return setup._run_bq_command(command, f\"Execute {sql_file_path}\")\n",
    "\n",
    "# Run SQL implementation\n",
    "run_sql_file()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ⚙️ Step 5: Generate Configuration File\n",
    "\n",
    "Create a configuration file for the analytics engine with all necessary settings."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ⚙️ Generate Configuration File\n",
    "def generate_config_file():\n",
    "    \"\"\"Generate configuration file for the analytics engine\"\"\"\n",
    "    config = {\n",
    "        'project_id': setup.project_id,\n",
    "        'dataset_location': setup.dataset_location,\n",
    "        'datasets': setup.datasets,\n",
    "        'vertex_ai_connection': 'vertex-connection',\n",
    "        'models': {\n",
    "            'multimodal_embedding_model': 'retail_models.multimodal_embedding_model',\n",
    "            'text_generation_model': 'retail_models.text_generation_model',\n",
    "            'vision_model': 'retail_models.vision_model'\n",
    "        },\n",
    "        'performance_targets': {\n",
    "            'query_timeout_seconds': 300,\n",
    "            'max_embeddings_batch_size': 100,\n",
    "            'vector_search_top_k': 10\n",
    "        }\n",
    "    }\n",
    "\n",
    "    config_path = Path('retail_analytics_config.yaml')\n",
    "    try:\n",
    "        with open(config_path, 'w') as f:\n",
    "            yaml.dump(config, f, default_flow_style=False)\n",
    "        logger.info(f\"Configuration file created: {config_path}\")\n",
    "        print(\"\\n\" + \"=\"*60)\n",
    "        print(\"⚙️ CONFIGURATION FILE GENERATED\")\n",
    "        print(\"=\"*60)\n",
    "        print(f\"📁 Config File: {config_path}\")\n",
    "        print(\"✅ Configuration file created successfully\")\n",
    "        return True\n",
    "    except Exception as e:\n",
    "        logger.error(f\"Failed to create config file: {str(e)}\")\n",
    "        print(f\"❌ Failed to create config file: {str(e)}\")\n",
    "        return False\n",
    "\n",
    "# Generate configuration file\n",
    "generate_config_file()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Step 6: Validate Setup\n",
    "\n",
    "Run validation checks to ensure all components are working correctly."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔍 Validate Setup\n",
    "def validate_setup():\n",
    "    \"\"\"Validate the setup by checking key components\"\"\"\n",
    "    logger.info(\"Validating setup...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🔍 VALIDATING SETUP\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    validation_results = {}\n",
    "\n",
    "    # Check datasets exist\n",
    "    for dataset in setup.datasets:\n",
    "        command = f\"show {dataset}\"\n",
    "        validation_results[f\"dataset_{dataset}\"] = setup._run_bq_command(\n",
    "            command, f\"Validate dataset {dataset} exists\"\n",
    "        )\n",
    "\n",
    "    # Check if we can run a simple query\n",
    "    test_query = \"SELECT 1 as test_value\"\n",
    "    command = f'query --use_legacy_sql=false \"{test_query}\"'\n",
    "    validation_results[\"basic_query\"] = setup._run_bq_command(\n",
    "        command, \"Test basic BigQuery query\"\n",
    "    )\n",
    "\n",
    "    # Summary\n",
    "    valid_components = sum(validation_results.values())\n",
    "    total_components = len(validation_results)\n",
    "    \n",
    "    print(f\"\\n📊 Validation Results: {valid_components}/{total_components}\")\n",
    "    for component, status in validation_results.items():\n",
    "        status_icon = \"✅\" if status else \"❌\"\n",
    "        print(f\"  {status_icon} {component.replace('_', ' ').title()}\")\n",
    "    \n",
    "    return validation_results\n",
    "\n",
    "# Run validation\n",
    "validation_results = validate_setup()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎉 Setup Summary\n",
    "\n",
    "### ✅ Setup Components Completed:\n",
    "\n",
    "1. **🗄️ Dataset Creation** - BigQuery datasets created\n",
    "2. **🔗 Vertex AI Connection** - ML model access configured\n",
    "3. **☁️ API Enablement** - Google Cloud APIs enabled\n",
    "4. **📄 SQL Implementation** - Tables, models, and functions created\n",
    "5. **⚙️ Configuration** - System configuration file generated\n",
    "6. **🔍 Validation** - Setup validation completed\n",
    "\n",
    "### 📊 System Architecture:\n",
    "```\n",
    "┌─────────────────────────────────────────────────────────────┐\n",
    "│                    BIGQUERY AI SYSTEM                        │\n",
    "├─────────────────────────────────────────────────────────────┤\n",
    "│  🗄️ retail_analytics     🧠 retail_models     📊 retail_insights │\n",
    "├─────────────────────────────────────────────────────────────┤\n",
    "│  📦 Products & Reviews    🤖 ML Models         📈 Analytics     │\n",
    "│  🧠 Embeddings           🔍 Vector Search     🎯 Insights       │\n",
    "│  📝 Sentiment Analysis   🎨 Multimodal        📋 Reports        │\n",
    "└─────────────────────────────────────────────────────────────┘\n",
    "```\n",
    "\n",
    "### 🚀 Next Steps:\n",
    "1. **Test the system** with the demo notebook\n",
    "2. **Run validation** with the test notebook\n",
    "3. **Deploy to production** if needed\n",
    "4. **Submit to Kaggle** competition\n",
    "\n",
    "### 🏆 Competition Ready:\n",
    "- ✅ **Complete BigQuery AI implementation**\n",
    "- ✅ **All three approaches** (Generative AI, Vector Search, Multimodal)\n",
    "- ✅ **Production-ready architecture**\n",
    "- ✅ **Live demo available**\n",
    "- ✅ **Enterprise-grade quality**\n",
    "\n",
    "**Your Intelligent Retail Analytics Engine is now fully operational!** 🎉\n",
    "\n",
    "**Ready to win $100,000 and launch your SaaS business!** 🚀💰"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🏗️ BigQuery AI: Intelligent Retail Analytics Engine Setup\n",
    "\n",
    "**Competition Entry**: BigQuery AI - Building the Future of Data\n",
    "**High-Quality Solution**: Enterprise-Grade Retail Intelligence\n",
    "**Author**: Senior Data Engineer & AI Architect\n",
    "\n",
    "---\n",
    "\n",
    "## 🎯 Overview\n",
    "\n",
    "This notebook provides a **complete setup guide** for deploying the Intelligent Retail Analytics Engine on Google Cloud Platform. The setup process includes:\n",
    "\n",
    "1. **🗄️ Dataset Creation** - BigQuery datasets and tables\n",
    "2. **🤖 Model Setup** - Vertex AI ML models\n",
    "3. **🔗 Connection Setup** - Vertex AI integration\n",
    "4. **📊 Data Population** - Sample data and embeddings\n",
    "5. **⚙️ Configuration** - System configuration files\n",
    "\n",
    "**Result**: Fully functional BigQuery AI retail analytics system"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📋 Prerequisites\n",
    "\n",
    "### Required Google Cloud Setup:\n",
    "1. **Google Cloud Project** with billing enabled\n",
    "2. **BigQuery API** enabled\n",
    "3. **Vertex AI API** enabled\n",
    "4. **Google Cloud SDK** installed and configured\n",
    "5. **Project Editor** or **Owner** permissions\n",
    "\n",
    "### Required Permissions:\n",
    "- `bigquery.datasets.create`\n",
    "- `bigquery.tables.create`\n",
    "- `bigquery.jobs.create`\n",
    "- `aiplatform.models.*`\n",
    "- `storage.objects.*`\n",
    "\n",
    "### Software Requirements:\n",
    "```bash\n",
    "# Install Google Cloud SDK\n",
    "curl https://sdk.cloud.google.com | bash\n",
    "exec -l $SHELL\n",
    "gcloud init\n",
    "\n",
    "# Install Python packages\n",
    "pip install google-cloud-bigquery google-cloud-aiplatform PyYAML\n",
    "```"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "{\n",
    " \"cells\": [\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"# 🏗️ BigQuery AI: Intelligent Retail Analytics Engine Setup\\n\",\n",
    "    \"\\n\",\n",
    "    \"**Competition Entry**: BigQuery AI - Building the Future of Data\\n\",\n",
    "    \"**High-Quality Solution**: Enterprise-Grade Retail Intelligence\\n\",\n",
    "    \"**Author**: Senior Data Engineer & AI Architect\\n\",\n",
    "    \"\\n\",\n",
    "    \"---\\n\",\n",
    "    \"\\n\",\n",
    "    \"## 🎯 Overview\\n\",\n",
    "    \"\\n\",\n",
    "    \"This notebook provides a **complete setup guide** for deploying the Intelligent Retail Analytics Engine on Google Cloud Platform. The setup process includes:\\n\",\n",
    "    \"\\n\",\n",
    "    \"1. **🗄️ Dataset Creation** - BigQuery datasets and tables\\n\",\n",
    "    \"2. **🤖 Model Setup** - Vertex AI ML models\\n\",\n",
    "    \"3. **🔗 Connection Setup** - Vertex AI integration\\n\",\n",
    "    \"4. **📊 Data Population** - Sample data and embeddings\\n\",\n",
    "    \"5. **⚙️ Configuration** - System configuration files\\n\",\n",
    "    \"\\n\",\n",
    "    \"**Result**: Fully functional BigQuery AI retail analytics system\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 📋 Prerequisites\\n\",\n",
    "    \"\\n\",\n",
    "    \"### Required Google Cloud Setup:\\n\",\n",
    "    \"1. **Google Cloud Project** with billing enabled\\n\",\n",
    "    \"2. **BigQuery API** enabled\\n\",\n",
    "    \"3. **Vertex AI API** enabled\\n\",\n",
    "    \"4. **Google Cloud SDK** installed and configured\\n\",\n",
    "    \"5. **Project Editor** or **Owner** permissions\\n\",\n",
    "    \"\\n\",\n",
    "    \"### Required Permissions:\\n\",\n",
    "    \"- `bigquery.datasets.create`\\n\",\n",
    "    \"- `bigquery.tables.create`\\n\",\n",
    "    \"- `bigquery.jobs.create`\\n\",\n",
    "    \"- `aiplatform.models.*`\\n\",\n",
    "    \"- `storage.objects.*`\\n\",\n",
    "    \"\\n\",\n",
    "    \"### Software Requirements:\\n\",\n",
    "    \"```bash\\n\",\n",
    "    \"# Install Google Cloud SDK\\n\",\n",
    "    \"curl https://sdk.cloud.google.com | bash\\n\",\n",
    "    \"exec -l $SHELL\\n\",\n",
    "    \"gcloud init\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Install Python packages\\n\",\n",
    "    \"pip install google-cloud-bigquery google-cloud-aiplatform PyYAML\\n\",\n",
    "    \"```\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 📦 Import required libraries\\n\",\n",
    "    \"import os\\n\",\n",
    "    \"import sys\\n\",\n",
    "    \"import time\\n\",\n",
    "    \"import logging\\n\",\n",
    "    \"from pathlib import Path\\n\",\n",
    "    \"from typing import Dict, List, Optional\\n\",\n",
    "    \"import yaml\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Configure logging\\n\",\n",
    "    \"logging.basicConfig(\\n\",\n",
    "    \"    level=logging.INFO,\\n\",\n",
    "    \"    format='%(asctime)s - %(levelname)s - %(message)s',\\n\",\n",
    "    \"    handlers=[\\n\",\n",
    "    \"        logging.FileHandler('retail_analytics_setup.log'),\\n\",\n",
    "    \"        logging.StreamHandler(sys.stdout)\\n\",\n",
    "    \"    ]\\n\",\n",
    "    \")\\n\",\n",
    "    \"logger = logging.getLogger(__name__)\\n\",\n",
    "    \"\\n\",\n",
    "    \"print(\\\"✅ Libraries imported successfully!\\\")\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 🔧 Configuration - UPDATE THESE VALUES\\n\",\n",
    "    \"PROJECT_ID = \\\"your-project-id\\\"  # Replace with your actual Google Cloud Project ID\\n\",\n",
    "    \"DATASET_LOCATION = \\\"us\\\"  # Choose: us, eu, asia, etc.\\n\",\n",
    "    \"\\n\",\n",
    "    \"print(f\\\"🔧 Project ID: {PROJECT_ID}\\\")\\n\",\n",
    "    \"print(f\\\"📍 Dataset Location: {DATASET_LOCATION}\\\")\\n\",\n",
    "    \"print(\\\"\\\\n📝 Make sure to update PROJECT_ID above with your actual Google Cloud Project ID\\\")\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 🏗️ Setup Class Definition\\n\",\n",
    "    \"class BigQueryRetailAnalyticsSetup:\\n\",\n",
    "    \"    \\\"\\\"\\\"Setup and deployment class for the Intelligent Retail Analytics Engine\\\"\\\"\\\"\\n\",\n",
    "    \"\\n\",\n",
    "    \"    def __init__(self, project_id: str, dataset_location: str = 'us'):\\n\",\n",
    "    \"        self.project_id = project_id\\n\",\n",
    "    \"        self.dataset_location = dataset_location\\n\",\n",
    "    \"        self.datasets = ['retail_analytics', 'retail_models', 'retail_insights']\\n\",\n",
    "    \"\\n\",\n",
    "    \"        # Check if gcloud is available\\n\",\n",
    "    \"        self._check_gcloud_setup()\\n\",\n",
    "    \"\\n\",\n",
    "    \"    def _check_gcloud_setup(self):\\n\",\n",
    "    \"        \\\"\\\"\\\"Verify Google Cloud SDK setup\\\"\\\"\\\"\\n\",\n",
    "    \"        try:\\n\",\n",
    "    \"            import subprocess\\n\",\n",
    "    \"            result = subprocess.run(['gcloud', '--version'],\\n\",\n",
    "    \"                                  capture_output=True, text=True, check=True)\\n\",\n",
    "    \"            logger.info(\\\"Google Cloud SDK found\\\")\\n\",\n",
    "    \"            print(\\\"✅ Google Cloud SDK found\\\")\\n\",\n",
    "    \"        except (subprocess.CalledProcessError, FileNotFoundError):\\n\",\n",
    "    \"            logger.error(\\\"Google Cloud SDK not found. Please install and configure gcloud CLI.\\\")\\n\",\n",
    "    \"            logger.error(\\\"Installation: https://cloud.google.com/sdk/docs/install\\\")\\n\",\n",
    "    \"            print(\\\"❌ Google Cloud SDK not found\\\")\\n\",\n",
    "    \"            print(\\\"Installation: https://cloud.google.com/sdk/docs/install\\\")\\n\",\n",
    "    \"            sys.exit(1)\\n\",\n",
    "    \"\\n\",\n",
    "    \"    def _run_bq_command(self, command: str, description: str) -> bool:\\n\",\n",
    "    \"        \\\"\\\"\\\"Execute BigQuery command with error handling\\\"\\\"\\\"\\n\",\n",
    "    \"        try:\\n\",\n",
    "    \"            logger.info(f\\\"Executing: {description}\\\")\\n\",\n",
    "    \"            import subprocess\\n\",\n",
    "    \"\\n\",\n",
    "    \"            # Add project ID to command if not present\\n\",\n",
    "    \"            if '--project_id' not in command and self.project_id not in command:\\n\",\n",
    "    \"                command = f\\\"bq --project_id={self.project_id} {command}\\\"\\n\",\n",
    "    \"\\n\",\n",
    "    \"            result = subprocess.run(command, shell=True, capture_output=True, text=True)\\n\",\n",
    "    \"\\n\",\n",
    "    \"            if result.returncode == 0:\\n\",\n",
    "    \"                logger.info(f\\\"✅ {description} completed successfully\\\")\\n\",\n",
    "    \"                print(f\\\"✅ {description} completed successfully\\\")\\n\",\n",
    "    \"                return True\\n\",\n",
    "    \"            else:\\n\",\n",
    "    \"                logger.error(f\\\"❌ {description} failed\\\")\\n\",\n",
    "    \"                logger.error(f\\\"Error: {result.stderr}\\\")\\n\",\n",
    "    \"                print(f\\\"❌ {description} failed\\\")\\n\",\n",
    "    \"                print(f\\\"Error: {result.stderr}\\\")\\n\",\n",
    "    \"                return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"        except Exception as e:\\n\",\n",
    "    \"            logger.error(f\\\"Exception during {description}: {str(e)}\\\")\\n\",\n",
    "    \"            print(f\\\"❌ Exception during {description}: {str(e)}\\\")\\n\",\n",
    "    \"            return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"    def _run_gcloud_command(self, command: str, description: str) -> bool:\\n\",\n",
    "    \"        \\\"\\\"\\\"Execute gcloud command with error handling\\\"\\\"\\\"\\n\",\n",
    "    \"        try:\\n\",\n",
    "    \"            logger.info(f\\\"Executing: {description}\\\")\\n\",\n",
    "    \"            import subprocess\\n\",\n",
    "    \"\\n\",\n",
    "    \"            result = subprocess.run(command, shell=True, capture_output=True, text=True)\\n\",\n",
    "    \"\\n\",\n",
    "    \"            if result.returncode == 0:\\n\",\n",
    "    \"                logger.info(f\\\"✅ {description} completed successfully\\\")\\n\",\n",
    "    \"                print(f\\\"✅ {description} completed successfully\\\")\\n\",\n",
    "    \"                return True\\n\",\n",
    "    \"            else:\\n\",\n",
    "    \"                logger.error(f\\\"❌ {description} failed\\\")\\n\",\n",
    "    \"                logger.error(f\\\"Error: {result.stderr}\\\")\\n\",\n",
    "    \"                print(f\\\"❌ {description} failed\\\")\\n\",\n",
    "    \"                print(f\\\"Error: {result.stderr}\\\")\\n\",\n",
    "    \"                return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"        except Exception as e:\\n\",\n",
    "    \"            logger.error(f\\\"Exception during {description}: {str(e)}\\\")\\n\",\n",
    "    \"            print(f\\\"❌ Exception during {description}: {str(e)}\\\")\\n\",\n",
    "    \"            return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Initialize setup\\n\",\n",
    "    \"setup = BigQueryRetailAnalyticsSetup(PROJECT_ID, DATASET_LOCATION)\\n\",\n",
    "    \"print(\\\"✅ Setup class initialized successfully!\\\")\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 🗄️ Step 1: Create BigQuery Datasets\\n\",\n",
    "    \"\\n\",\n",
    "    \"Create the required BigQuery datasets for the retail analytics system.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 🗄️ Create BigQuery Datasets\\n\",\n",
    "    \"def create_datasets():\\n\",\n",
    "    \"    \\\"\\\"\\\"Create required BigQuery datasets\\\"\\\"\\\"\\n\",\n",
    "    \"    logger.info(\\\"Creating BigQuery datasets...\\\")\\n\",\n",
    "    \"    print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"    print(\\\"🗄️ CREATING BIGQUERY DATASETS\\\")\\n\",\n",
    "    \"    print(\\\"=\\\"*60)\\n\",\n",
    "    \"\\n\",\n",
    "    \"    success_count = 0\\n\",\n",
    "    \"    for dataset in setup.datasets:\\n\",\n",
    "    \"        dataset_id = f\\\"{setup.project_id}:{dataset}\\\"\\n\",\n",
    "    \"        command = f\\\"mk --dataset --location={setup.dataset_location} {dataset_id}\\\"\\n\",\n",
    "    \"\\n\",\n",
    "    \"        if setup._run_bq_command(command, f\\\"Create dataset {dataset}\\\"):\\n\",\n",
    "    \"            success_count += 1\\n\",\n",
    "    \"        else:\\n\",\n",
    "    \"            logger.warning(f\\\"Dataset {dataset} might already exist\\\")\\n\",\n",
    "    \"            print(f\\\"⚠️ Dataset {dataset} might already exist\\\")\\n\",\n",
    "    \"\\n\",\n",
    "    \"    logger.info(f\\\"Created {success_count}/{len(setup.datasets)} datasets\\\")\\n\",\n",
    "    \"    print(f\\\"\\\\n📊 Dataset Creation: {success_count}/{len(setup.datasets)} completed\\\")\\n\",\n",
    "    \"    return success_count > 0\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Run dataset creation\\n\",\n",
    "    \"create_datasets()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 🔗 Step 2: Setup Vertex AI Connection\\n\",\n",
    "    \"\\n\",\n",
    "    \"Create a Vertex AI connection for BigQuery ML to access Vertex AI models.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 🔗 Setup Vertex AI Connection\\n\",\n",
    "    \"def setup_vertex_ai_connection():\\n\",\n",
    "    \"    \\\"\\\"\\\"Set up Vertex AI connection for BigQuery ML\\\"\\\"\\\"\\n\",\n",
    "    \"    logger.info(\\\"Setting up Vertex AI connection...\\\")\\n\",\n",
    "    \"    print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"    print(\\\"🔗 SETTING UP VERTEX AI CONNECTION\\\")\\n\",\n",
    "    \"    print(\\\"=\\\"*60)\\n\",\n",
    "    \"\\n\",\n",
    "    \"    connection_name = \\\"vertex-connection\\\"\\n\",\n",
    "    \"    command = f\\\"mk --connection --connection_type=CLOUD_RESOURCE --location={setup.dataset_location} {connection_name}\\\"\\n\",\n",
    "    \"\\n\",\n",
    "    \"    if setup._run_bq_command(command, \\\"Create Vertex AI connection\\\"):\\n\",\n",
    "    \"        logger.info(\\\"Vertex AI connection created successfully\\\")\\n\",\n",
    "    \"        print(\\\"✅ Vertex AI connection created successfully\\\")\\n\",\n",
    "    \"        print(\\\"\\\\n📝 Note: You may need to grant the BigQuery Connection Service Account access to Vertex AI\\\")\\n\",\n",
    "    \"        return True\\n\",\n",
    "    \"    else:\\n\",\n",
    "    \"        logger.warning(\\\"Vertex AI connection setup may have failed\\\")\\n\",\n",
    "    \"        print(\\\"⚠️ Vertex AI connection setup may have failed\\\")\\n\",\n",
    "    \"        return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Run Vertex AI connection setup\\n\",\n",
    "    \"setup_vertex_ai_connection()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## ☁️ Step 3: Enable Required APIs\\n\",\n",
    "    \"\\n\",\n",
    "    \"Enable the necessary Google Cloud APIs for the system to function.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# ☁️ Enable Required APIs\\n\",\n",
    "    \"def enable_required_apis():\\n\",\n",
    "    \"    \\\"\\\"\\\"Enable required Google Cloud APIs\\\"\\\"\\\"\\n\",\n",
    "    \"    logger.info(\\\"Enabling required Google Cloud APIs...\\\")\\n\",\n",
    "    \"    print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"    print(\\\"☁️ ENABLING REQUIRED GOOGLE CLOUD APIs\\\")\\n\",\n",
    "    \"    print(\\\"=\\\"*60)\\n\",\n",
    "    \"\\n\",\n",
    "    \"    apis = [\\n\",\n",
    "    \"        'bigquery.googleapis.com',\\n\",\n",
    "    \"        'bigqueryconnection.googleapis.com',\\n\",\n",
    "    \"        'aiplatform.googleapis.com'\\n\",\n",
    "    \"    ]\\n\",\n",
    "    \"\\n\",\n",
    "    \"    success_count = 0\\n\",\n",
    "    \"    for api in apis:\\n\",\n",
    "    \"        command = f\\\"services enable {api}\\\"\\n\",\n",
    "    \"        if setup._run_gcloud_command(command, f\\\"Enable {api}\\\"):\\n\",\n",
    "    \"            success_count += 1\\n\",\n",
    "    \"\\n\",\n",
    "    \"    logger.info(f\\\"Enabled {success_count}/{len(apis)} APIs\\\")\\n\",\n",
    "    \"    print(f\\\"\\\\n📊 API Enablement: {success_count}/{len(apis)} completed\\\")\\n\",\n",
    "    \"    return success_count == len(apis)\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Run API enablement\\n\",\n",
    "    \"enable_required_apis()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 📄 Step 4: Execute SQL Implementation\\n\",\n",
    "    \"\\n\",\n",
    "    \"Run the main SQL implementation file to create all tables, models, and functions.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 📄 Execute SQL Implementation\\n\",\n",
    "    \"def run_sql_file():\\n\",\n",
    "    \"    \\\"\\\"\\\"Execute SQL file in BigQuery\\\"\\\"\\\"\\n\",\n",
    "    \"    sql_file_path = \\\"../retail_analytics_engine.sql\\\"\\n\",\n",
    "    \"    \\n\",\n",
    "    \"    if not Path(sql_file_path).exists():\\n\",\n",
    "    \"        logger.error(f\\\"SQL file not found: {sql_file_path}\\\")\\n\",\n",
    "    \"        print(f\\\"❌ SQL file not found: {sql_file_path}\\\")\\n\",\n",
    "    \"        return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"    logger.info(f\\\"Executing SQL file: {sql_file_path}\\\")\\n\",\n",
    "    \"    print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"    print(\\\"📄 EXECUTING SQL IMPLEMENTATION\\\")\\n\",\n",
    "    \"    print(\\\"=\\\"*60)\\n\",\n",
    "    \"    print(f\\\"📁 SQL File: {sql_file_path}\\\")\\n\",\n",
    "    \"\\n\",\n",
    "    \"    command = f\\\"query --use_legacy_sql=false < {sql_file_path}\\\"\\n\",\n",
    "    \"\\n\",\n",
    "    \"    return setup._run_bq_command(command, f\\\"Execute {sql_file_path}\\\")\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Run SQL implementation\\n\",\n",
    "    \"run_sql_file()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## ⚙️ Step 5: Generate Configuration File\\n\",\n",
    "    \"\\n\",\n",
    "    \"Create a configuration file for the analytics engine with all necessary settings.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# ⚙️ Generate Configuration File\\n\",\n",
    "    \"def generate_config_file():\\n\",\n",
    "    \"    \\\"\\\"\\\"Generate configuration file for the analytics engine\\\"\\\"\\\"\\n\",\n",
    "    \"    config = {\\n\",\n",
    "    \"        'project_id': setup.project_id,\\n\",\n",
    "    \"        'dataset_location': setup.dataset_location,\\n\",\n",
    "    \"        'datasets': setup.datasets,\\n\",\n",
    "    \"        'vertex_ai_connection': 'vertex-connection',\\n\",\n",
    "    \"        'models': {\\n\",\n",
    "    \"            'multimodal_embedding_model': 'retail_models.multimodal_embedding_model',\\n\",\n",
    "    \"            'text_generation_model': 'retail_models.text_generation_model',\\n\",\n",
    "    \"            'vision_model': 'retail_models.vision_model'\\n\",\n",
    "    \"        },\\n\",\n",
    "    \"        'performance_targets': {\\n\",\n",
    "    \"            'query_timeout_seconds': 300,\\n\",\n",
    "    \"            'max_embeddings_batch_size': 100,\\n\",\n",
    "    \"            'vector_search_top_k': 10\\n\",\n",
    "    \"        }\\n\",\n",
    "    \"    }\\n\",\n",
    "    \"\\n\",\n",
    "    \"    config_path = Path('retail_analytics_config.yaml')\\n\",\n",
    "    \"    try:\\n\",\n",
    "    \"        with open(config_path, 'w') as f:\\n\",\n",
    "    \"            yaml.dump(config, f, default_flow_style=False)\\n\",\n",
    "    \"        logger.info(f\\\"Configuration file created: {config_path}\\\")\\n\",\n",
    "    \"        print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"        print(\\\"⚙️ CONFIGURATION FILE GENERATED\\\")\\n\",\n",
    "    \"        print(\\\"=\\\"*60)\\n\",\n",
    "    \"        print(f\\\"📁 Config File: {config_path}\\\")\\n\",\n",
    "    \"        print(\\\"✅ Configuration file created successfully\\\")\\n\",\n",
    "    \"        return True\\n\",\n",
    "    \"    except Exception as e:\\n\",\n",
    "    \"        logger.error(f\\\"Failed to create config file: {str(e)}\\\")\\n\",\n",
    "    \"        print(f\\\"❌ Failed to create config file: {str(e)}\\\")\\n\",\n",
    "    \"        return False\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Generate configuration file\\n\",\n",
    "    \"generate_config_file()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 🔍 Step 6: Validate Setup\\n\",\n",
    "    \"\\n\",\n",
    "    \"Run validation checks to ensure all components are working correctly.\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"code\",\n",
    "   \"execution_count\": null,\n",
    "   \"metadata\": {},\n",
    "   \"outputs\": [],\n",
    "   \"source\": [\n",
    "    \"# 🔍 Validate Setup\\n\",\n",
    "    \"def validate_setup():\\n\",\n",
    "    \"    \\\"\\\"\\\"Validate the setup by checking key components\\\"\\\"\\\"\\n\",\n",
    "    \"    logger.info(\\\"Validating setup...\\\")\\n\",\n",
    "    \"    print(\\\"\\\\n\\\" + \\\"=\\\"*60)\\n\",\n",
    "    \"    print(\\\"🔍 VALIDATING SETUP\\\")\\n\",\n",
    "    \"    print(\\\"=\\\"*60)\\n\",\n",
    "    \"\\n\",\n",
    "    \"    validation_results = {}\\n\",\n",
    "    \"\\n\",\n",
    "    \"    # Check datasets exist\\n\",\n",
    "    \"    for dataset in setup.datasets:\\n\",\n",
    "    \"        command = f\\\"show {dataset}\\\"\\n\",\n",
    "    \"        validation_results[f\\\"dataset_{dataset}\\\"] = setup._run_bq_command(\\n\",\n",
    "    \"            command, f\\\"Validate dataset {dataset} exists\\\"\\n\",\n",
    "    \"        )\\n\",\n",
    "    \"\\n\",\n",
    "    \"    # Check if we can run a simple query\\n\",\n",
    "    \"    test_query = \\\"SELECT 1 as test_value\\\"\\n\",\n",
    "    \"    command = f'query --use_legacy_sql=false \\\"{test_query}\\\"'\\n\",\n",
    "    \"    validation_results[\\\"basic_query\\\"] = setup._run_bq_command(\\n\",\n",
    "    \"        command, \\\"Test basic BigQuery query\\\"\\n\",\n",
    "    \"    )\\n\",\n",
    "    \"\\n\",\n",
    "    \"    # Summary\\n\",\n",
    "    \"    valid_components = sum(validation_results.values())\\n\",\n",
    "    \"    total_components = len(validation_results)\\n\",\n",
    "    \"    \\n\",\n",
    "    \"    print(f\\\"\\\\n📊 Validation Results: {valid_components}/{total_components}\\\")\\n\",\n",
    "    \"    for component, status in validation_results.items():\\n\",\n",
    "    \"        status_icon = \\\"✅\\\" if status else \\\"❌\\\"\\n\",\n",
    "    \"        print(f\\\"  {status_icon} {component.replace('_', ' ').title()}\\\")\\n\",\n",
    "    \"    \\n\",\n",
    "    \"    return validation_results\\n\",\n",
    "    \"\\n\",\n",
    "    \"# Run validation\\n\",\n",
    "    \"validation_results = validate_setup()\"\n",
    "   ]\n",
    "  },\n",
    "  {\n",
    "   \"cell_type\": \"markdown\",\n",
    "   \"metadata\": {},\n",
    "   \"source\": [\n",
    "    \"## 🎉 Setup Summary\\n\",\n",
    "    \"\\n\",\n",
    "    \"### ✅ Setup Components Completed:\\n\",\n",
    "    \"\\n\",\n",
    "    \"1. **🗄️ Dataset Creation** - BigQuery datasets created\\n\",\n",
    "    \"2. **🔗 Vertex AI Connection** - ML model access configured\\n\",\n",
    "    \"3. **☁️ API Enablement** - Google Cloud APIs enabled\\n\",\n",
    "    \"4. **📄 SQL Implementation** - Tables, models, and functions created\\n\",\n",
    "    \"5. **⚙️ Configuration** - System configuration file generated\\n\",\n",
    "    \"6. **🔍 Validation** - Setup validation completed\\n\",\n",
    "    \"\\n\",\n",
    "    \"### 📊 System Architecture:\\n\",\n",
    "    \"```\\n\",\n",
    "    \"┌─────────────────────────────────────────────────────────────┐\\n\",\n",
    "    \"│                    BIGQUERY AI SYSTEM                        │\\n\",\n",
    "    \"├─────────────────────────────────────────────────────────────┤\\n\",\n",
    "    \"│  🗄️ retail_analytics     🧠 retail_models     📊 retail_insights │\\n\",\n",
    "    \"├─────────────────────────────────────────────────────────────┤\\n\",\n",
    "    \"│  📦 Products & Reviews    🤖 ML Models         📈 Analytics     │\\n\",\n",
    "    \"│  🧠 Embeddings           🔍 Vector Search     🎯 Insights       │\\n\",\n",
    "    \"│  📝 Sentiment Analysis   🎨 Multimodal        📋 Reports        │\\n\",\n",
    "    \"└─────────────────────────────────────────────────────────────┘\\n\",\n",
    "    \"```\\n\",\n",
    "    \"\\n\",\n",
    "    \"### 🚀 Next Steps:\\n\",\n",
    "    \"1. **Test the system** with the demo notebook\\n\",\n",
    "    \"2. **Run validation** with the test notebook\\n\",\n",
    "    \"3. **Deploy to production** if needed\\n\",\n",
    "    \"4. **Submit to Kaggle** competition\\n\",\n",
    "    \"\\n\",\n",
    "    \"### 🏆 Competition Ready:\\n\",\n",
    "    \"- ✅ **Complete BigQuery AI implementation**\\n\",\n",
    "    \"- ✅ **All three approaches** (Generative AI, Vector Search, Multimodal)\\n\",\n",
    "    \"- ✅ **Production-ready architecture**\\n\",\n",
    "    \"- ✅ **Live demo available**\\n\",\n",
    "    \"- ✅ **Enterprise-grade quality**\\n\",\n",
    "    \"\\n\",\n",
    "    \"**Your Intelligent Retail Analytics Engine is now fully operational!** 🎉\\n\",\n",
    "    \"\\n\",\n",
    "    \"**Ready to win $100,000 and launch your SaaS business!** 🚀💰\"\n",
    "   ]\n",
    "  }\n",
    " ],\n",
    " \"metadata\": {\n",
    "  \"kernelspec\": {\n",
    "   \"display_name\": \"Python 3 (ipykernel)\",\n",
    "   \"language\": \"python\",\n",
    "   \"name\": \"python3\"\n",
    "  },\n",
    "  \"language_info\": {\n",
    "   \"codemirror_mode\": {\n",
    "    \"name\": \"ipython\",\n",
    "    \"version\": 3\n",
    "   },\n",
    "   \"file_extension\": \".py\",\n",
    "   \"mimetype\": \"text/x-python\",\n",
    "   \"name\": \"python\",\n",
    "   \"nbconvert_exporter\": \"python\",\n",
    "   \"pygments_lexer\": \"ipython3\",\n",
    "   \"version\": \"3.13.7\"\n",
    "  }\n",
    " },\n",
    " \"nbformat\": 4,\n",
    " \"nbformat_minor\": 4\n",
    "}\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔧 Configuration - UPDATE THESE VALUES\n",
    "PROJECT_ID = \"intelligent-retail-analytics\"  # Replace with your actual Google Cloud Project ID\n",
    "DATASET_LOCATION = \"us\"  # Choose: us, eu, asia, etc.\n",
    "\n",
    "print(f\"🔧 Project ID: {PROJECT_ID}\")\n",
    "print(f\"📍 Dataset Location: {DATASET_LOCATION}\")\n",
    "print(\"\\n📝 Make sure to update PROJECT_ID above with your actual Google Cloud Project ID\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🏗️ Setup Class Definition\n",
    "class BigQueryRetailAnalyticsSetup:\n",
    "    \"\"\"Setup and deployment class for the Intelligent Retail Analytics Engine\"\"\"\n",
    "\n",
    "    def __init__(self, project_id: str, dataset_location: str = 'us'):\n",
    "        self.project_id = project_id\n",
    "        self.dataset_location = dataset_location\n",
    "        self.datasets = ['retail_analytics', 'retail_models', 'retail_insights']\n",
    "\n",
    "        # Check if gcloud is available\n",
    "        self._check_gcloud_setup()\n",
    "\n",
    "    def _check_gcloud_setup(self):\n",
    "        \"\"\"Verify Google Cloud SDK setup\"\"\"\n",
    "        try:\n",
    "            import subprocess\n",
    "            result = subprocess.run(['gcloud', '--version'],\n",
    "                                  capture_output=True, text=True, check=True)\n",
    "            logger.info(\"Google Cloud SDK found\")\n",
    "            print(\"✅ Google Cloud SDK found\")\n",
    "        except (subprocess.CalledProcessError, FileNotFoundError):\n",
    "            logger.error(\"Google Cloud SDK not found. Please install and configure gcloud CLI.\")\n",
    "            logger.error(\"Installation: https://cloud.google.com/sdk/docs/install\")\n",
    "            print(\"❌ Google Cloud SDK not found\")\n",
    "            print(\"Installation: https://cloud.google.com/sdk/docs/install\")\n",
    "            sys.exit(1)\n",
    "\n",
    "    def _run_bq_command(self, command: str, description: str) -> bool:\n",
    "        \"\"\"Execute BigQuery command with error handling\"\"\"\n",
    "        try:\n",
    "            logger.info(f\"Executing: {description}\")\n",
    "            import subprocess\n",
    "\n",
    "            # Add project ID to command if not present\n",
    "            if '--project_id' not in command and self.project_id not in command:\n",
    "                command = f\"bq --project_id={self.project_id} {command}\"\n",
    "\n",
    "            result = subprocess.run(command, shell=True, capture_output=True, text=True)\n",
    "\n",
    "            if result.returncode == 0:\n",
    "                logger.info(f\"✅ {description} completed successfully\")\n",
    "                print(f\"✅ {description} completed successfully\")\n",
    "                return True\n",
    "            else:\n",
    "                logger.error(f\"❌ {description} failed\")\n",
    "                logger.error(f\"Error: {result.stderr}\")\n",
    "                print(f\"❌ {description} failed\")\n",
    "                print(f\"Error: {result.stderr}\")\n",
    "                return False\n",
    "\n",
    "        except Exception as e:\n",
    "            logger.error(f\"Exception during {description}: {str(e)}\")\n",
    "            print(f\"❌ Exception during {description}: {str(e)}\")\n",
    "            return False\n",
    "\n",
    "    def _run_gcloud_command(self, command: str, description: str) -> bool:\n",
    "        \"\"\"Execute gcloud command with error handling\"\"\"\n",
    "        try:\n",
    "            logger.info(f\"Executing: {description}\")\n",
    "            import subprocess\n",
    "\n",
    "            result = subprocess.run(command, shell=True, capture_output=True, text=True)\n",
    "\n",
    "            if result.returncode == 0:\n",
    "                logger.info(f\"✅ {description} completed successfully\")\n",
    "                print(f\"✅ {description} completed successfully\")\n",
    "                return True\n",
    "            else:\n",
    "                logger.error(f\"❌ {description} failed\")\n",
    "                logger.error(f\"Error: {result.stderr}\")\n",
    "                print(f\"❌ {description} failed\")\n",
    "                print(f\"Error: {result.stderr}\")\n",
    "                return False\n",
    "\n",
    "        except Exception as e:\n",
    "            logger.error(f\"Exception during {description}: {str(e)}\")\n",
    "            print(f\"❌ Exception during {description}: {str(e)}\")\n",
    "            return False\n",
    "\n",
    "# Initialize setup\n",
    "setup = BigQueryRetailAnalyticsSetup(PROJECT_ID, DATASET_LOCATION)\n",
    "print(\"✅ Setup class initialized successfully!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🗄️ Step 1: Create BigQuery Datasets\n",
    "\n",
    "Create the required BigQuery datasets for the retail analytics system."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🗄️ Create BigQuery Datasets\n",
    "def create_datasets():\n",
    "    \"\"\"Create required BigQuery datasets\"\"\"\n",
    "    logger.info(\"Creating BigQuery datasets...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🗄️ CREATING BIGQUERY DATASETS\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    success_count = 0\n",
    "    for dataset in setup.datasets:\n",
    "        dataset_id = f\"{setup.project_id}:{dataset}\"\n",
    "        command = f\"mk --dataset --location={setup.dataset_location} {dataset_id}\"\n",
    "\n",
    "        if setup._run_bq_command(command, f\"Create dataset {dataset}\"):\n",
    "            success_count += 1\n",
    "        else:\n",
    "            logger.warning(f\"Dataset {dataset} might already exist\")\n",
    "            print(f\"⚠️ Dataset {dataset} might already exist\")\n",
    "\n",
    "    logger.info(f\"Created {success_count}/{len(setup.datasets)} datasets\")\n",
    "    print(f\"\\n📊 Dataset Creation: {success_count}/{len(setup.datasets)} completed\")\n",
    "    return success_count > 0\n",
    "\n",
    "# Run dataset creation\n",
    "create_datasets()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔗 Step 2: Setup Vertex AI Connection\n",
    "\n",
    "Create a Vertex AI connection for BigQuery ML to access Vertex AI models."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔗 Setup Vertex AI Connection\n",
    "def setup_vertex_ai_connection():\n",
    "    \"\"\"Set up Vertex AI connection for BigQuery ML\"\"\"\n",
    "    logger.info(\"Setting up Vertex AI connection...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🔗 SETTING UP VERTEX AI CONNECTION\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    connection_name = \"vertex-connection\"\n",
    "    command = f\"mk --connection --connection_type=CLOUD_RESOURCE --location={setup.dataset_location} {connection_name}\"\n",
    "\n",
    "    if setup._run_bq_command(command, \"Create Vertex AI connection\"):\n",
    "        logger.info(\"Vertex AI connection created successfully\")\n",
    "        print(\"✅ Vertex AI connection created successfully\")\n",
    "        print(\"\\n📝 Note: You may need to grant the BigQuery Connection Service Account access to Vertex AI\")\n",
    "        return True\n",
    "    else:\n",
    "        logger.warning(\"Vertex AI connection setup may have failed\")\n",
    "        print(\"⚠️ Vertex AI connection setup may have failed\")\n",
    "        return False\n",
    "\n",
    "# Run Vertex AI connection setup\n",
    "setup_vertex_ai_connection()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ☁️ Step 3: Enable Required APIs\n",
    "\n",
    "Enable the necessary Google Cloud APIs for the system to function."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ☁️ Enable Required APIs\n",
    "def enable_required_apis():\n",
    "    \"\"\"Enable required Google Cloud APIs\"\"\"\n",
    "    logger.info(\"Enabling required Google Cloud APIs...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"☁️ ENABLING REQUIRED GOOGLE CLOUD APIs\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    apis = [\n",
    "        'bigquery.googleapis.com',\n",
    "        'bigqueryconnection.googleapis.com',\n",
    "        'aiplatform.googleapis.com'\n",
    "    ]\n",
    "\n",
    "    success_count = 0\n",
    "    for api in apis:\n",
    "        command = f\"services enable {api}\"\n",
    "        if setup._run_gcloud_command(command, f\"Enable {api}\"):\n",
    "            success_count += 1\n",
    "\n",
    "    logger.info(f\"Enabled {success_count}/{len(apis)} APIs\")\n",
    "    print(f\"\\n📊 API Enablement: {success_count}/{len(apis)} completed\")\n",
    "    return success_count == len(apis)\n",
    "\n",
    "# Run API enablement\n",
    "enable_required_apis()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 📄 Step 4: Execute SQL Implementation\n",
    "\n",
    "Run the main SQL implementation file to create all tables, models, and functions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 📄 Execute SQL Implementation\n",
    "def run_sql_file():\n",
    "    \"\"\"Execute SQL file in BigQuery\"\"\"\n",
    "    sql_file_path = \"retail_analytics_engine.sql\"\n",
    "    \n",
    "    if not Path(sql_file_path).exists():\n",
    "        logger.error(f\"SQL file not found: {sql_file_path}\")\n",
    "        print(f\"❌ SQL file not found: {sql_file_path}\")\n",
    "        return False\n",
    "\n",
    "    logger.info(f\"Executing SQL file: {sql_file_path}\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"📄 EXECUTING SQL IMPLEMENTATION\")\n",
    "    print(\"=\"*60)\n",
    "    print(f\"📁 SQL File: {sql_file_path}\")\n",
    "\n",
    "    command = f\"query --use_legacy_sql=false < {sql_file_path}\"\n",
    "\n",
    "    return setup._run_bq_command(command, f\"Execute {sql_file_path}\")\n",
    "\n",
    "# Run SQL implementation\n",
    "run_sql_file()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## ⚙️ Step 5: Generate Configuration File\n",
    "\n",
    "Create a configuration file for the analytics engine with all necessary settings."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# ⚙️ Generate Configuration File\n",
    "def generate_config_file():\n",
    "    \"\"\"Generate configuration file for the analytics engine\"\"\"\n",
    "    config = {\n",
    "        'project_id': setup.project_id,\n",
    "        'dataset_location': setup.dataset_location,\n",
    "        'datasets': setup.datasets,\n",
    "        'vertex_ai_connection': 'vertex-connection',\n",
    "        'models': {\n",
    "            'multimodal_embedding_model': 'retail_models.multimodal_embedding_model',\n",
    "            'text_generation_model': 'retail_models.text_generation_model',\n",
    "            'vision_model': 'retail_models.vision_model'\n",
    "        },\n",
    "        'performance_targets': {\n",
    "            'query_timeout_seconds': 300,\n",
    "            'max_embeddings_batch_size': 100,\n",
    "            'vector_search_top_k': 10\n",
    "        }\n",
    "    }\n",
    "\n",
    "    config_path = Path('retail_analytics_config.yaml')\n",
    "    try:\n",
    "        with open(config_path, 'w') as f:\n",
    "            yaml.dump(config, f, default_flow_style=False)\n",
    "        logger.info(f\"Configuration file created: {config_path}\")\n",
    "        print(\"\\n\" + \"=\"*60)\n",
    "        print(\"⚙️ CONFIGURATION FILE GENERATED\")\n",
    "        print(\"=\"*60)\n",
    "        print(f\"📁 Config File: {config_path}\")\n",
    "        print(\"✅ Configuration file created successfully\")\n",
    "        return True\n",
    "    except Exception as e:\n",
    "        logger.error(f\"Failed to create config file: {str(e)}\")\n",
    "        print(f\"❌ Failed to create config file: {str(e)}\")\n",
    "        return False\n",
    "\n",
    "# Generate configuration file\n",
    "generate_config_file()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🔍 Step 6: Validate Setup\n",
    "\n",
    "Run validation checks to ensure all components are working correctly."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 🔍 Validate Setup\n",
    "def validate_setup():\n",
    "    \"\"\"Validate the setup by checking key components\"\"\"\n",
    "    logger.info(\"Validating setup...\")\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"🔍 VALIDATING SETUP\")\n",
    "    print(\"=\"*60)\n",
    "\n",
    "    validation_results = {}\n",
    "\n",
    "    # Check datasets exist\n",
    "    for dataset in setup.datasets:\n",
    "        command = f\"show {dataset}\"\n",
    "        validation_results[f\"dataset_{dataset}\"] = setup._run_bq_command(\n",
    "            command, f\"Validate dataset {dataset} exists\"\n",
    "        )\n",
    "\n",
    "    # Check if we can run a simple query\n",
    "    test_query = \"SELECT 1 as test_value\"\n",
    "    command = f'query --use_legacy_sql=false \"{test_query}\"'\n",
    "    validation_results[\"basic_query\"] = setup._run_bq_command(\n",
    "        command, \"Test basic BigQuery query\"\n",
    "    )\n",
    "\n",
    "    # Summary\n",
    "    valid_components = sum(validation_results.values())\n",
    "    total_components = len(validation_results)\n",
    "    \n",
    "    print(f\"\\n📊 Validation Results: {valid_components}/{total_components}\")\n",
    "    for component, status in validation_results.items():\n",
    "        status_icon = \"✅\" if status else \"❌\"\n",
    "        print(f\"  {status_icon} {component.replace('_', ' ').title()}\")\n",
    "    \n",
    "    return validation_results\n",
    "\n",
    "# Run validation\n",
    "validation_results = validate_setup()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 🎉 Setup Summary\n",
    "\n",
    "### ✅ Setup Components Completed:\n",
    "\n",
    "1. **🗄️ Dataset Creation** - BigQuery datasets created\n",
    "2. **🔗 Vertex AI Connection** - ML model access configured\n",
    "3. **☁️ API Enablement** - Google Cloud APIs enabled\n",
    "4. **📄 SQL Implementation** - Tables, models, and functions created\n",
    "5. **⚙️ Configuration** - System configuration file generated\n",
    "6. **🔍 Validation** - Setup validation completed\n",
    "\n",
    "### 📊 System Architecture:\n",
    "```\n",
    "┌─────────────────────────────────────────────────────────────┐\n",
    "│                    BIGQUERY AI SYSTEM                        │\n",
    "├─────────────────────────────────────────────────────────────┤\n",
    "│  🗄️ retail_analytics     🧠 retail_models     📊 retail_insights │\n",
    "├─────────────────────────────────────────────────────────────┤\n",
    "│  📦 Products & Reviews    🤖 ML Models         📈 Analytics     │\n",
    "│  🧠 Embeddings           🔍 Vector Search     🎯 Insights       │\n",
    "│  📝 Sentiment Analysis   🎨 Multimodal        📋 Reports        │\n",
    "└─────────────────────────────────────────────────────────────┘\n",
    "```\n",
    "\n",
    "### 🚀 Next Steps:\n",
    "1. **Test the system** with the demo notebook\n",
    "2. **Run validation** with the test notebook\n",
    "3. **Deploy to production** if needed\n",
    "4. **Submit to Kaggle** competition\n",
    "\n",
    "### 🏆 Competition Ready:\n",
    "- ✅ **Complete BigQuery AI implementation**\n",
    "- ✅ **All three approaches** (Generative AI, Vector Search, Multimodal)\n",
    "- ✅ **Production-ready architecture**\n",
    "- ✅ **Live demo available**\n",
    "- ✅ **Enterprise-grade quality**\n",
    "\n",
    "**Your Intelligent Retail Analytics Engine is now fully operational!** 🎉\n",
    "\n",
    "**Ready to win $100,000 and launch your SaaS business!** 🚀💰"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}


In [ ]:
# 🏗️ Setup Class Definition
class BigQueryRetailAnalyticsSetup:
    """Setup and deployment class for the Intelligent Retail Analytics Engine"""

    def __init__(self, project_id: str, dataset_location: str = 'us'):
        self.project_id = project_id
        self.dataset_location = dataset_location
        self.datasets = ['retail_analytics', 'retail_models', 'retail_insights']

        # Check if gcloud is available
        self._check_gcloud_setup()

    def _check_gcloud_setup(self):
        """Verify Google Cloud SDK setup"""
        try:
            import subprocess
            result = subprocess.run(['gcloud', '--version'],
                                  capture_output=True, text=True, check=True)
            logger.info("Google Cloud SDK found")
            print("✅ Google Cloud SDK found")
        except (subprocess.CalledProcessError, FileNotFoundError):
            logger.error("Google Cloud SDK not found. Please install and configure gcloud CLI.")
            logger.error("Installation: https://cloud.google.com/sdk/docs/install")
            print("❌ Google Cloud SDK not found")
            print("Installation: https://cloud.google.com/sdk/docs/install")
            sys.exit(1)

    def _run_bq_command(self, command: str, description: str) -> bool:
        """Execute BigQuery command with error handling"""
        try:
            logger.info(f"Executing: {description}")
            import subprocess

            # Add project ID to command if not present
            if '--project_id' not in command and self.project_id not in command:
                command = f"bq --project_id={self.project_id} {command}"

            result = subprocess.run(command, shell=True, capture_output=True, text=True)

            if result.returncode == 0:
                logger.info(f"✅ {description} completed successfully")
                print(f"✅ {description} completed successfully")
                return True
            else:
                logger.error(f"❌ {description} failed")
                logger.error(f"Error: {result.stderr}")
                print(f"❌ {description} failed")
                print(f"Error: {result.stderr}")
                return False

        except Exception as e:
            logger.error(f"Exception during {description}: {str(e)}")
            print(f"❌ Exception during {description}: {str(e)}")
            return False

    def _run_gcloud_command(self, command: str, description: str) -> bool:
        """Execute gcloud command with error handling"""
        try:
            logger.info(f"Executing: {description}")
            import subprocess

            result = subprocess.run(command, shell=True, capture_output=True, text=True)

            if result.returncode == 0:
                logger.info(f"✅ {description} completed successfully")
                print(f"✅ {description} completed successfully")
                return True
            else:
                logger.error(f"❌ {description} failed")
                logger.error(f"Error: {result.stderr}")
                print(f"❌ {description} failed")
                print(f"Error: {result.stderr}")
                return False

        except Exception as e:
            logger.error(f"Exception during {description}: {str(e)}")
            print(f"❌ Exception during {description}: {str(e)}")
            return False

# Initialize setup
setup = BigQueryRetailAnalyticsSetup(PROJECT_ID, DATASET_LOCATION)
print("✅ Setup class initialized successfully!")

## 🗄️ Step 1: Create BigQuery Datasets

Create the required BigQuery datasets for the retail analytics system.

In [ ]:
# 🗄️ Create BigQuery Datasets
def create_datasets():
    """Create required BigQuery datasets"""
    logger.info("Creating BigQuery datasets...")
    print("\n" + "="*60)
    print("🗄️ CREATING BIGQUERY DATASETS")
    print("="*60)

    success_count = 0
    for dataset in setup.datasets:
        dataset_id = f"{setup.project_id}:{dataset}"
        command = f"mk --dataset --location={setup.dataset_location} {dataset_id}"

        if setup._run_bq_command(command, f"Create dataset {dataset}"):
            success_count += 1
        else:
            logger.warning(f"Dataset {dataset} might already exist")
            print(f"⚠️ Dataset {dataset} might already exist")

    logger.info(f"Created {success_count}/{len(setup.datasets)} datasets")
    print(f"\n📊 Dataset Creation: {success_count}/{len(setup.datasets)} completed")
    return success_count > 0

# Run dataset creation
create_datasets()

## 🔗 Step 2: Setup Vertex AI Connection

Create a Vertex AI connection for BigQuery ML to access Vertex AI models.

In [ ]:
# 🔗 Setup Vertex AI Connection
def setup_vertex_ai_connection():
    """Set up Vertex AI connection for BigQuery ML"""
    logger.info("Setting up Vertex AI connection...")
    print("\n" + "="*60)
    print("🔗 SETTING UP VERTEX AI CONNECTION")
    print("="*60)

    connection_name = "vertex-connection"
    command = f"mk --connection --connection_type=CLOUD_RESOURCE --location={setup.dataset_location} {connection_name}"

    if setup._run_bq_command(command, "Create Vertex AI connection"):
        logger.info("Vertex AI connection created successfully")
        print("✅ Vertex AI connection created successfully")
        print("\n📝 Note: You may need to grant the BigQuery Connection Service Account access to Vertex AI")
        return True
    else:
        logger.warning("Vertex AI connection setup may have failed")
        print("⚠️ Vertex AI connection setup may have failed")
        return False

# Run Vertex AI connection setup
setup_vertex_ai_connection()

## ☁️ Step 3: Enable Required APIs

Enable the necessary Google Cloud APIs for the system to function.

In [ ]:
# ☁️ Enable Required APIs
def enable_required_apis():
    """Enable required Google Cloud APIs"""
    logger.info("Enabling required Google Cloud APIs...")
    print("\n" + "="*60)
    print("☁️ ENABLING REQUIRED GOOGLE CLOUD APIs")
    print("="*60)

    apis = [
        'bigquery.googleapis.com',
        'bigqueryconnection.googleapis.com',
        'aiplatform.googleapis.com'
    ]

    success_count = 0
    for api in apis:
        command = f"services enable {api}"
        if setup._run_gcloud_command(command, f"Enable {api}"):
            success_count += 1

    logger.info(f"Enabled {success_count}/{len(apis)} APIs")
    print(f"\n📊 API Enablement: {success_count}/{len(apis)} completed")
    return success_count == len(apis)

# Run API enablement
enable_required_apis()

## 📄 Step 4: Execute SQL Implementation

Run the main SQL implementation file to create all tables, models, and functions.

In [ ]:
# 📄 Execute SQL Implementation
def run_sql_file():
    """Execute SQL file in BigQuery"""
    sql_file_path = "retail_analytics_engine.sql"
    
    if not Path(sql_file_path).exists():
        logger.error(f"SQL file not found: {sql_file_path}")
        print(f"❌ SQL file not found: {sql_file_path}")
        return False

    logger.info(f"Executing SQL file: {sql_file_path}")
    print("\n" + "="*60)
    print("📄 EXECUTING SQL IMPLEMENTATION")
    print("="*60)
    print(f"📁 SQL File: {sql_file_path}")

    command = f"query --use_legacy_sql=false < {sql_file_path}"

    return setup._run_bq_command(command, f"Execute {sql_file_path}")

# Run SQL implementation
run_sql_file()

## ⚙️ Step 5: Generate Configuration File

Create a configuration file for the analytics engine with all necessary settings.

In [ ]:
# ⚙️ Generate Configuration File
def generate_config_file():
    """Generate configuration file for the analytics engine"""
    config = {
        'project_id': setup.project_id,
        'dataset_location': setup.dataset_location,
        'datasets': setup.datasets,
        'vertex_ai_connection': 'vertex-connection',
        'models': {
            'multimodal_embedding_model': 'retail_models.multimodal_embedding_model',
            'text_generation_model': 'retail_models.text_generation_model',
            'vision_model': 'retail_models.vision_model'
        },
        'performance_targets': {
            'query_timeout_seconds': 300,
            'max_embeddings_batch_size': 100,
            'vector_search_top_k': 10
        }
    }

    config_path = Path('retail_analytics_config.yaml')
    try:
        with open(config_path, 'w') as f:
            yaml.dump(config, f, default_flow_style=False)
        logger.info(f"Configuration file created: {config_path}")
        print("\n" + "="*60)
        print("⚙️ CONFIGURATION FILE GENERATED")
        print("="*60)
        print(f"📁 Config File: {config_path}")
        print("✅ Configuration file created successfully")
        return True
    except Exception as e:
        logger.error(f"Failed to create config file: {str(e)}")
        print(f"❌ Failed to create config file: {str(e)}")
        return False

# Generate configuration file
generate_config_file()

## 🔍 Step 6: Validate Setup

Run validation checks to ensure all components are working correctly.

In [ ]:
# 🔍 Validate Setup
def validate_setup():
    """Validate the setup by checking key components"""
    logger.info("Validating setup...")
    print("\n" + "="*60)
    print("🔍 VALIDATING SETUP")
    print("="*60)

    validation_results = {}

    # Check datasets exist
    for dataset in setup.datasets:
        command = f"show {dataset}"
        validation_results[f"dataset_{dataset}"] = setup._run_bq_command(
            command, f"Validate dataset {dataset} exists"
        )

    # Check if we can run a simple query
    test_query = "SELECT 1 as test_value"
    command = f'query --use_legacy_sql=false "{test_query}"'
    validation_results["basic_query"] = setup._run_bq_command(
        command, "Test basic BigQuery query"
    )

    # Summary
    valid_components = sum(validation_results.values())
    total_components = len(validation_results)
    
    print(f"\n📊 Validation Results: {valid_components}/{total_components}")
    for component, status in validation_results.items():
        status_icon = "✅" if status else "❌"
        print(f"  {status_icon} {component.replace('_', ' ').title()}")
    
    return validation_results

# Run validation
validation_results = validate_setup()

## 🎉 Setup Summary

### ✅ Setup Components Completed:

1. **🗄️ Dataset Creation** - BigQuery datasets created
2. **🔗 Vertex AI Connection** - ML model access configured
3. **☁️ API Enablement** - Google Cloud APIs enabled
4. **📄 SQL Implementation** - Tables, models, and functions created
5. **⚙️ Configuration** - System configuration file generated
6. **🔍 Validation** - Setup validation completed

### 📊 System Architecture:
```
┌─────────────────────────────────────────────────────────────┐
│                    BIGQUERY AI SYSTEM                        │
├─────────────────────────────────────────────────────────────┤
│  🗄️ retail_analytics     🧠 retail_models     📊 retail_insights │
├─────────────────────────────────────────────────────────────┤
│  📦 Products & Reviews    🤖 ML Models         📈 Analytics     │
│  🧠 Embeddings           🔍 Vector Search     🎯 Insights       │
│  📝 Sentiment Analysis   🎨 Multimodal        📋 Reports        │
└─────────────────────────────────────────────────────────────┘
```

### 🚀 Next Steps:
1. **Test the system** with the demo notebook
2. **Run validation** with the test notebook
3. **Deploy to production** if needed
4. **Submit to Kaggle** competition

### 🏆 Competition Ready:
- ✅ **Complete BigQuery AI implementation**
- ✅ **All three approaches** (Generative AI, Vector Search, Multimodal)
- ✅ **Production-ready architecture**
- ✅ **Live demo available**
- ✅ **Enterprise-grade quality**

**Your Intelligent Retail Analytics Engine is now fully operational!** 🎉

**Ready to win $100,000 and launch your SaaS business!** 🚀💰